Observations


(1)Observed the names and merged the categories that I feel to be merged.

(2)For text cleaning I have tested results with/without stopwords, with/without numerics . 'without' gives good results.

(3)I have tested 3 columns individually : headline,description, authors. also checked combination of twos and found: combined all three is giving best result.



In [181]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, precision_recall_fscore_support
import csv

In [168]:
stopwards = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [169]:
file =pd.read_json("/content/drive/MyDrive/News_Category_Dataset_v2.json",lines=True,orient='columns')

#concat 3 columns and consider them as input column
file['concated'] = file['headline'] + ' ' + file['short_description'] + ' ' + file['authors']
del file['link'],file['date'],file['headline'],file['short_description'],file['authors']

In [170]:
#
file.loc[file.category.isin(['CULTURE & ARTS','ARTS & CULTURE','ARTS']), 'category'] = 'ARTS'
file.loc[file.category.isin(['EDUCATION','COLLEGE']), 'category'] = 'EDUCATION'
file.loc[file.category.isin(['THE WORLDPOST','WORLDPOST','WORLD NEWS']), 'category'] = 'WORLDPOST'
file.loc[file.category.isin(['STYLE & BEAUTY','STYLE']), 'category'] = 'STYLE'

In [171]:
# Text Cleaning
def text_preprocess(text):
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stopwards]
    text = ' '.join(text)
    text = re.sub(r'[^\w\s]', '', text)
    text= re.sub('[0-9\n]','',text)
    text = text.strip()
    text = re.sub(' +', ' ', text)    
    return(text)
file['concated'] = file['concated'].apply(text_preprocess)

In [172]:
file.head()

,category,concated
0,CRIME,mass shootings texas last week tv left husband...
1,ENTERTAINMENT,smith joins diplo nicky jam world cups officia...
2,ENTERTAINMENT,hugh grant marries first time age actor longti...
3,ENTERTAINMENT,jim carrey blasts castrato adam schiff democra...
4,ENTERTAINMENT,julianna margulies uses donald trump poop bags...


<h1>fasttext</h1>

In [173]:
! pip install fasttext
import fasttext

In [174]:
file['category_fasttext'] = '__label__' + file['category'].astype(str)
X_train, X_test, y_train, y_test = train_test_split(file['concated'], file['category_fasttext'], test_size=0.25, random_state=42)

In [175]:
traindf = pd.DataFrame()
traindf['x'] = X_train
traindf['y'] = y_train
traindf.to_csv(r'pairings.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [176]:
model = fasttext.train_supervised(input="pairings.txt")

In [177]:
prediction = []
for i in X_test:
    pred = model.predict(i)[0][0]
    prediction.append(pred)

In [178]:
confusion_matrix(y_test, prediction)

array([[ 470,    0,   12, ...,   52,   24,   21],
       [  37,    0,   19, ...,   30,   25,   11],
       [   6,    0,  702, ...,  134,   21,   35],
       ...,
       [  10,    0,   56, ..., 3682,   26,    9],
       [  16,    0,   24, ...,  118,  292,    8],
       [  14,    0,   38, ...,   41,   15, 1407]])

In [179]:
precision_recall_fscore_support(y_test, prediction, average='macro')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.3731514051259989, 0.39238981542323237, 0.37424043630871684, None)

In [180]:
print(classification_report(y_test, prediction))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

                         precision    recall  f1-score   support

          __label__ARTS       0.45      0.48      0.46       982
  __label__BLACK VOICES       0.00      0.00      0.00      1113
      __label__BUSINESS       0.43      0.49      0.46      1427
        __label__COMEDY       0.43      0.47      0.45      1325
         __label__CRIME       0.49      0.59      0.54       788
       __label__DIVORCE       0.78      0.72      0.75       883
     __label__EDUCATION       0.45      0.39      0.42       529
 __label__ENTERTAINMENT       0.56      0.74      0.63      3937
   __label__ENVIRONMENT       0.51      0.20      0.29       343
         __label__FIFTY       0.14      0.05      0.07       342
  __label__FOOD & DRINK       0.00      0.00      0.00      1569
     __label__GOOD NEWS       0.00      0.00      0.00       341
         __label__GREEN       0.34      0.38      0.36       642
__label__HEALTHY LIVING       0.00      0.00      0.00      1651
 __label__HOME & LIVING 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
